1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 => 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 청크가 가질 수 있는 크기
    chunk_overlap=200, # 청크를 쪼갤때 겹치는 범위 => 유사도 검색을 진행할 때 성공 확률을 높이기 위해
)
loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name= 'tax-index'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pc=Pinecone(api_key = pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
query='연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query) # 유사도 검사

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=database.as_retriever(), chain_type_kwargs={
        "prompt": prompt
    }
)

In [ ]:
ai_message = qa_chain({
    "query": query
})

In [ ]:
ai_message